In [29]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [30]:
!ls ../input

In [31]:
import torch
from torch import nn
from torchvision import datasets, models, transforms
import torch.nn.functional as F

In [32]:
import torch.utils.data as tdata

In [33]:
!ls ../input/seg_train/seg_train/
!ls ../input/seg_test/seg_test/

In [34]:
data_path_format = '../input/seg_{0}/seg_{0}'

In [35]:
np.random.seed(5315)
torch.manual_seed(9784)

In [36]:
# channel_means = (0.4301, 0.4572, 0.4540)
# channel_stds = (0.2698, 0.2680, 0.2983)  # 0.485, 0.456, 0.406
channel_means = (0.485, 0.456, 0.406)
channel_stds = (0.229, 0.224, 0.225)

In [37]:
image_transforms = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),
    transforms.Normalize(channel_means, channel_stds)
])

In [38]:
image_datasets = dict(zip(('dev', 'test'), [datasets.ImageFolder(data_path_format.format(key),transform=image_transforms) for key in ['train', 'test']]))
print(image_datasets)

In [39]:
devset_indices = np.arange(len(image_datasets['dev']))
devset_labels = image_datasets['dev'].targets

In [40]:
from sklearn import model_selection

In [41]:
train_indices, val_indices, train_labels,  val_labels = model_selection.train_test_split(devset_indices, devset_labels, test_size=0.1, stratify=devset_labels)

In [42]:
print(len(val_indices))

In [43]:
image_datasets['train'] = tdata.Subset(image_datasets['dev'], train_indices)
image_datasets['validation'] = tdata.Subset(image_datasets['dev'], val_indices)

In [44]:
print(image_datasets)

In [45]:
from IPython.display import display

In [46]:
image_dataloaders = {key: tdata.DataLoader(image_datasets[key], batch_size=16,shuffle=True) for key in  ['train', 'validation']}

In [47]:
image_dataloaders['test'] = tdata.DataLoader(image_datasets['test'], batch_size=32)

In [48]:
def imshow(inp, title=None, fig_size=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0)) # C x H x W  # H x W x C
    inp = channel_stds * inp + channel_means
    inp = np.clip(inp, 0, 1)
    fig = plt.figure(figsize=fig_size)
    ax = fig.add_subplot(111)
    ax.imshow(inp)
    if title is not None:
        ax.set_title(title)
    ax.set_aspect('equal')
    plt.pause(0.001)

In [49]:
imshow(image_datasets['train'][8555][0]) # 5946

In [50]:
cuda_device = torch.device('cuda')
cpu_device = torch.device('cpu')
device = cuda_device

In [51]:
class Lab3Net(torch.nn.Module):
    def __init__(self):
        super(Lab3Net, self).__init__()
        self.batch_norm000 = torch.nn.BatchNorm2d(3)

        self.conv00 = torch.nn.Conv2d(3, 16, 3, padding=1)
        self.act00  = torch.nn.ReLU()
        self.batch_norm00 = torch.nn.BatchNorm2d(16)
        self.pool00 = torch.nn.MaxPool2d(2, 2)
        
        self.conv0 = torch.nn.Conv2d(16, 32, 3, padding=1)
        self.act0  = torch.nn.ReLU()
        self.batch_norm0 = torch.nn.BatchNorm2d(32)
        self.pool0 = torch.nn.MaxPool2d(2, 2)
        
        self.conv1 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.act1  = torch.nn.ReLU()
        self.batch_norm1 = torch.nn.BatchNorm2d(64)
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        
        self.conv2 = torch.nn.Conv2d(64, 128, 3, padding=1)
        self.act2  = torch.nn.ReLU()
        self.batch_norm2 = torch.nn.BatchNorm2d(128)
        self.pool2 = torch.nn.MaxPool2d(2, 2)
        
        self.conv3 = torch.nn.Conv2d(128, 256, 3, padding=1)
        self.act3  = torch.nn.ReLU()
        self.batch_norm3 = torch.nn.BatchNorm2d(256)
        self.pool3 = torch.nn.MaxPool2d(2, 2)

        self.fc1   = torch.nn.Linear(4096, 256)
        self.act4  = torch.nn.Tanh()
        self.batch_norm4 = torch.nn.BatchNorm1d(256)
        
        self.fc2   = torch.nn.Linear(256, 64)
        self.act5  = torch.nn.Tanh()
        self.batch_norm5 = torch.nn.BatchNorm1d(64)
        
        self.fc3   = torch.nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.batch_norm000(x)
        x = self.conv00(x)
        x = self.act00(x)
        x = self.batch_norm00(x)
        x = self.pool00(x)
        
        x = self.conv0(x)
        x = self.act0(x)
        x = self.batch_norm0(x)
        x = self.pool0(x)
        
        x = self.conv1(x)
        x = self.act1(x)
        x = self.batch_norm1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.act2(x)
        x = self.batch_norm2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.act3(x)
        x = self.batch_norm3(x)
        x = self.pool3(x)
        
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        x = self.act4(x)
        x = self.batch_norm4(x)
        x = self.fc2(x)
        x = self.act5(x)
        x = self.batch_norm5(x)
        x = self.fc3(x)
        
        return x


ptr = Lab3Net()
#num_ftrs = ptr.fc.in_features
#ptr.fc = nn.Linear(num_ftrs, 6)
ptr = ptr.to(device)

In [52]:
print(torch.cuda.is_available())
torch.cuda.device_count()
import sys
print(sys.version)

In [53]:
class BestModel:
    def __init__(self, path, initial_criterion):
        self.path = path
        self.criterion = initial_criterion
        
    def update(self, model, optimizer, criterion):
        self.criterion = criterion
        torch.save({'model_state': model.state_dict(), 'optimizer_state': optimizer.state_dict(), 'criterion': criterion}, self.path)
        
    def load_model_data(self):
        return torch.load(self.path)
    
    def restore(self, model, optimizer):
        model_data = self.load_model_data()
        model.load_state_dict(model_data['model_state'])
        optimizer.load_state_dict(model_data['optimizer_state'])

In [54]:
def train_model(epochs, model, optimizer, criterion, loaders, device, best_model, n_prints=1):
    print_every = len(loaders['train']) // n_prints
    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        
        for iteration, (xx, yy) in enumerate(loaders['train']):
            optimizer.zero_grad()
            xx, yy = xx.to(device), yy.to(device)
            out = model(xx)
            loss = criterion(out, yy)
            running_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            
            if(iteration % print_every == print_every - 1):
                running_train_loss /= print_every
                print(f"Epoch {epoch}, iteration {iteration} training_loss {running_train_loss}")
                running_train_loss = 0.0
            
        with torch.no_grad():
            model.eval()
            running_corrects = 0
            running_total = 0
            running_loss = 0.0
            for xx, yy in loaders['validation']:
                batch_size = xx.size(0)
                xx, yy = xx.to(device), yy.to(device)

                out = model(xx)
                
                loss = criterion(out, yy)
                running_loss += loss.item()
                
                predictions = out.argmax(1)
                running_corrects += (predictions == yy).sum().item()
                running_total += batch_size
            
            mean_val_loss = running_loss / len(loaders['validation'])
            accuracy = running_corrects / running_total
            
            if accuracy > best_model.criterion:
                best_model.update(model, optimizer, accuracy)
            
            print(f"Epoch {epoch}, val_loss {mean_val_loss}, accuracy = {accuracy}")
    best_model.restore(model, optimizer)

In [55]:
import time
start = time.time()

In [56]:
optimizer = torch.optim.Adam(ptr.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss()
best_model = BestModel('resnet_data',0.0)
train_model(20, ptr, optimizer, criterion,image_dataloaders, device, best_model=best_model, n_prints=5)

In [61]:
from torchvision.utils import make_grid

In [57]:
end = time.time()
print(end - start)

In [62]:
xx, yy = next(iter(image_dataloaders['validation']))

In [63]:
ptr.eval()
out = ptr(xx.cuda())
preds = out.argmax(1)
predicted_classes = [image_datasets['dev'].classes[c] for c in preds]
actual_classes = [image_datasets['dev'].classes[c] for c in yy]
imshow(make_grid(xx, nrow=4), fig_size=(8,8))
print(predicted_classes)
print(actual_classes)

In [64]:
all_preds = []
correct_preds = []
ptr.eval()
for xx, yy in image_dataloaders['test']:
    xx = xx.to(device)
    output = ptr(xx)
    all_preds.extend(output.argmax(1).tolist())
    correct_preds.extend(yy.tolist())

In [66]:
from sklearn import metrics

In [67]:
all_preds = np.asarray(all_preds)
correct_preds = np.asarray(correct_preds)

In [68]:
target_names = image_datasets['test'].classes
print(metrics.classification_report(correct_preds, all_preds,target_names=target_names))

In [69]:
confusion_matrix = metrics.confusion_matrix(correct_preds, all_preds)
pd.DataFrame(confusion_matrix, index=target_names, columns=target_names)